In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
# os.environ['CUDA_LAUNCH_BLOCKING']='1'
import random
import torchaudio
from torchaudio import transforms

from torchsummary import summary
import gc
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score


from torchvision import models
from tqdm import tqdm

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'

config = {
    'epochs': 100,
    'batch_size' : 32,
    'context' : 48,
    'learning_rate' : 0.01,
    'architecture' : 'very-low-cutoff'
}

In [3]:
class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, data_path, am_path, gender = "female", phoneme_idx = 4, am_idx = 1, MAX_LEN = 128, partition = "train"):
        """
        :param data_path: the root path of phonemes
        :param am_path: the path of am (.csv)
        :param gender: female or male
        :param phoneme_idx: the phoneme index
        :param am_idx: the index of target AM, should be int within [1, 96]
        :param MAX_LEN: max length of voice seq, if less, pad, if more, slice
        :param partition: train / val1 / val2 / test
        """

        self.MAX_LEN = MAX_LEN
        # get phoneme list
        self.target_phoneme_path = "/".join([data_path, gender, str(int(phoneme_idx))])
        phoneme_list = sorted(os.listdir(self.target_phoneme_path))
        random.shuffle(phoneme_list)
        length = len(phoneme_list)
        if partition == "train":
            self.phoneme_list = phoneme_list[:int(0.7 * length)]
        elif partition == "val1":
            self.phoneme_list = phoneme_list[int(0.7 * length):int(0.8 * length)]
        elif partition == "val2":
            self.phoneme_list = phoneme_list[int(0.8 * length):int(0.9 * length)]
        elif partition == "test":
            self.phoneme_list = phoneme_list[int(0.9 * length):]
            

        self.length = len(self.phoneme_list)

        # get_am data
        am_data = pd.read_csv(am_path)
        self.am_data = am_data[["ID", str(am_idx)]]

    def __len__(self):
        return self.length

    def spectro_gram(self, sig, n_mels=64, n_fft=1024, hop_len=None):
        top_db = 80

        # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
        spec = transforms.MelSpectrogram(44100, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

        # Convert to decibels
        spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
        return spec

    def __getitem__(self, ind):
        item_filename = self.phoneme_list[ind]
        item_full_path = "/".join([self.target_phoneme_path, item_filename])
        phoneme = np.load(item_full_path)

        person_id = int(item_filename.split("_")[0][1:7])
        try:
            target_am = self.am_data[self.am_data["ID"] == person_id].values[0][-1]
        except:
            print("person id =", person_id)
            target_am = 0.

        # padding
        phoneme = torch.tensor(phoneme, dtype=torch.float) #.reshape(1, -1)
        # apply mel transform
        phoneme = self.spectro_gram(phoneme)

        std, mean = torch.std_mean(phoneme, unbiased=False, dim=0)
        phoneme = (phoneme - mean) / (std + 1e-6)

        if len(phoneme[0]) < MAX_LEN:
            phoneme = np.pad(phoneme, ((0, 0), (0, MAX_LEN - len(phoneme[0]))), 'constant', constant_values=(0, 0))
            phoneme = torch.from_numpy(phoneme)
        else:
            phoneme = phoneme[:, :MAX_LEN]
        # phoneme = torch.from_numpy(phoneme)
        ##################################################################
        phoneme.unsqueeze_(0)
        ##################################################################
        target_am = torch.tensor(target_am).to(torch.float32)
        
        return phoneme, target_am


In [4]:
def train(model, optimizer, criterion, dataloader):

    model.train()
    train_loss = 0.0 #Monitoring Loss
    
    #########################################################
    # AM_true_list = []
    # AM_pred_list = []
    #########################################################
    
    for iter, (phoneme, AM) in enumerate(dataloader):
        scheduler.step()
        ### Move Data to Device (Ideally GPU)
        phoneme = phoneme.to(device)
        AM = AM.to(device)

        ### Forward Propagation
        preds_AM = model(phoneme)

        ### Loss Calculation
        # print(AM.shape)
        preds_AM = torch.squeeze(preds_AM)
        # print(preds_AM)
        # print(preds_AM.shape)model = models.shufflenet_v2_x1_0(weights=None).to(device)
        loss = criterion(preds_AM, AM)
        train_loss += loss.item()
        
        #########################################################
        ### Store Pred and True Labels
        # AM_pred_list.extend(preds_AM.cpu().tolist())
        # AM_true_list.extend(AM.cpu().tolist())
        #########################################################

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()
        # if iter % 20 == 0:
        #     print("iter =", iter, "loss =",loss.item())
    train_loss /= len(dataloader)
    print("Learning rate = ", scheduler.get_last_lr()[0])
    print("Train loss = ", train_loss)
    
    #########################################################
    # print(AM_pred_list)
    # print(AM_true_list)
    # print(len(AM_pred_list))
    # print(len(AM_true_list))
    # accuracy = mean_squared_error(AM_pred_list, AM_true_list)
    # print("Train MSE accuracy: ", accuracy)
    #########################################################
    
    # scheduler.step() # add schedule learning rate
    return train_loss

In [5]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode

    AM_true_list = []
    AM_pred_list = []

    for i, data in enumerate(dataloader):

        phoneme, AM = data
        ### Move data to device (ideally GPU)
        phoneme, AM = phoneme.to(device), AM.to(device) 

        with torch.inference_mode(): # makes sure that there are no gradients computed as we are not training the model now
            ### Forward Propagation
            ### Get Predictions
            predicted_AM = model(phoneme)
            # print(predicted_AM)
        
        ### Store Pred and True Labels
        AM_pred_list.extend(predicted_AM.cpu().tolist())
        AM_true_list.extend(AM.cpu().tolist())
        
        # Do you think we need loss.backward() and optimizer.step() here?
    
        del phoneme, AM, predicted_AM
        torch.cuda.empty_cache()

    ###############################################################################################
    # print(AM_pred_list[1000:3100])
    # print(AM_true_list)
    # print(len(AM_pred_list))
    # print(len(AM_true_list))
    ###############################################################################################
    
    # print("Number of equals between two list: ", sum(a == b for a,b in zip(AM_pred_list, AM_true_list)))
    
    ### Calculate Accuracy
    MSE = mean_squared_error(AM_pred_list, AM_true_list)
    r2_score_acc = r2_score(AM_pred_list, AM_true_list)
    MAE = mean_absolute_error(AM_pred_list, AM_true_list)
    print("Validation r2_score: ", r2_score_acc)
    print("Validation MAE: ", MAE)
    
    return MSE

In [6]:
def test(model, test_loader):
  ### What you call for model to perform inference?
    model.eval()

  ### List to store predicted phonemes of test data
    test_predictions = []
    ground_truth = []

  ### Which mode do you need to avoid gradients?
    with torch.inference_mode():

        for i, data in enumerate(tqdm(test_loader)):

            phoneme, groundtruth_AM = data
            ### Move data to device (ideally GPU)
            phoneme, groundtruth_AM = phoneme.to(device), groundtruth_AM.to(device)         
          
            predicted_AM = model(phoneme)
            predicted_AM.squeeze_()
            # print(predicted_AM.shape)
            # print(groundtruth_AM.shape)

          ### How do you store predicted_phonemes with test_predictions? Hint, look at eval 
            test_predictions.extend(predicted_AM.cpu().tolist())
            ground_truth.extend(groundtruth_AM.cpu())
    
    # print(len(test_predictions))
    return test_predictions, ground_truth

In [ ]:
# default_root_path = "./penstate_data/extract_phoneme"
default_root_path = "./penstate_data/extract_phoneme_processed"

# am_path = "./penstate_data/AMs_unnormalized.csv"
am_path = "./penstate_data/AMs_final.csv"

############## Female ##################
gender = "female"


# am_list = [89, 13, 88, 51, 14]
# phoneme_list = [7, 4, 31, 17, 6]
index = 0
am_list = [89, 13, 88, 51, 14]
phoneme_list = [31, 17, 6]

if gender == "female":
    g_flag = "F"
else:
    g_flag = "M"

for j in am_list:
    for k in phoneme_list:
        phoneme_idx = k
        am_idx = j
        folder_name = "%s_"%g_flag +"phoneme%s_"%phoneme_idx +  "AM%s"%am_idx
        os.mkdir(folder_name)
        for x in range(10):
            
            # default_root_path = "./penstate_data/extract_phoneme"
            default_root_path = "./penstate_data/extract_phoneme_processed"

            # am_path = "./penstate_data/AMs_unnormalized.csv"
            am_path = "./penstate_data/AMs_final.csv"

            gender = "female"
            
            
            MAX_LEN = 32 # TODO: may be too small
            batch_size = 64
            batch_size = config['batch_size']
            train_data = AudioDataset(data_path=default_root_path,
                                        am_path = am_path,
                                        gender = gender, phoneme_idx = phoneme_idx, am_idx = am_idx, MAX_LEN = MAX_LEN, partition="train")

            val_data = AudioDataset(data_path=default_root_path,
                                        am_path = am_path,
                                        gender = gender, phoneme_idx = phoneme_idx, am_idx = am_idx, MAX_LEN = MAX_LEN, partition="val1")
            test_data = AudioDataset(data_path=default_root_path,
                                        am_path = am_path,
                                        gender = gender, phoneme_idx = phoneme_idx, am_idx = am_idx, MAX_LEN = MAX_LEN, partition="val1")

            train_loader = torch.utils.data.DataLoader(train_data, num_workers=0,
                                                           batch_size=batch_size, shuffle=True)

            val_loader = torch.utils.data.DataLoader(val_data, num_workers=0,
                                                           batch_size=batch_size)
            test_loader = torch.utils.data.DataLoader(test_data, num_workers=0,
                                                           batch_size=batch_size)
            # generate txt
            all_am = None
            for i, data in enumerate(train_loader):
                phoneme, target_am = data
                # sns.heatmap(phoneme[0], cmap="rainbow")
                # plt.show()
                if all_am is None:
                    all_am = target_am
                else:
                    all_am = torch.cat([all_am, target_am])
                # print(phoneme.shape, target_am.shape)
                # break
            with open("./%s"%folder_name +"/%s"%folder_name + ".txt", "a+") as f:
                f.write(f'{phoneme_idx},{am_idx},{all_am.mean().item()}\n')
            print(f'{phoneme_idx},{am_idx},{all_am.mean().item()}\n')
            
            
            model = models.mnasnet1_0(weights=None).to(device)
            # print(model)
            model.layers[0] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            model.classifier[1] = nn.Linear(in_features=1280, out_features=1, bias=True)
            model = model.to(device)
            phoneme, AM = next(iter(train_loader))
                       
                
            criterion = torch.nn.MSELoss() #Defining Loss function 
            optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate']) #Defining Optimizer
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * config['epochs']))
            
            # Iterate over number of epochs to train and evaluate your model
            torch.cuda.empty_cache()
            gc.collect()
            
            
            best_mse = 1.0 ### Monitor best accuracy in your run

            for epoch in range(config['epochs']):
                print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

                train_loss = train(model, optimizer, criterion, train_loader)
                MSE = eval(model, val_loader)

                print("\tTrain Loss: ", train_loss)
                print("\tValidation MSE: ", MSE)

                ### Save checkpoint if accuracy is better than your current best
                if MSE < best_mse:
                    best_mse = MSE
                ### Save checkpoint with information you want
                    torch.save({'epoch': epoch,
                          'model_state_dict': model.state_dict(),
                          'optimizer_state_dict': optimizer.state_dict(),
                          'loss': train_loss,
                          'learning rate': scheduler.get_last_lr()[0],
                          'mse': MSE}, 
                    './model_checkpoint.pth')

            # predict the result 
            predictions, ground_truth = test(model, test_loader)
            print(len(test_loader))
            print(len(predictions))
            ### Create CSV file with predictions
            # print(x)
            with open("./%s"%folder_name +"/%s"%folder_name + "_times%s"%x + ".csv", "w+") as f:
                f.write("person, label, prediction\n")
                for i in range(len(predictions)):
                    f.write("{},{},{}\n".format(i, ground_truth[i], predictions[i]))



31,89,-0.3414126932621002


Epoch 1/100


/home/oscar/anaconda3/envs/torch_project/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Learning rate =  0.009997532801828659
Train loss =  1.6647136211395264
Validation r2_score:  -5800050270397.289
Validation MAE:  1.6419519037008286
	Train Loss:  1.6647136211395264
	Validation MSE:  2.7826575009194543

Epoch 2/100
Learning rate =  0.00999013364214136
Train loss =  15.702658653259277
Validation r2_score:  -4067704373935.422
Validation MAE:  1.6961964089423418
	Train Loss:  15.702658653259277
	Validation MSE:  3.038501751784498

Epoch 3/100
Learning rate =  0.009977809823015401
Train loss =  10.083882331848145
Validation r2_score:  -6087790417240.226
Validation MAE:  1.7418692807356517
	Train Loss:  10.083882331848145
	Validation MSE:  3.2949925793622157

Epoch 4/100
Learning rate =  0.009960573506572392
Train loss =  18.48908042907715
Validation r2_score:  -837342529131811.2
Validation MAE:  1.7032285891473293
	Train Loss:  18.48908042907715
	Validation MSE:  3.0755489272498298

Epoch 5/100
Learning rate =  0.00993844170297569
Train loss =  7.6560821533203125
Validation

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 212.81it/s]

1
3


...

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 129.60it/s]

1
3


17,89,-0.12616342306137085


Epoch 1/100


/home/oscar/anaconda3/envs/torch_project/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Learning rate =  0.009997532801828656
Train loss =  1.3686441741483186
Validation r2_score:  -115380786133.54337
Validation MAE:  0.8100606992019963
	Train Loss:  1.3686441741483186
	Validation MSE:  1.0477622611799353

Epoch 2/100
Learning rate =  0.009990133642141354
Train loss =  1.0496115583565928
Validation r2_score:  -602204775402.6893
Validation MAE:  0.8118388761810956
	Train Loss:  1.0496115583565928
	Validation MSE:  1.0438759125329427

Epoch 3/100
Learning rate =  0.009977809823015393
Train loss =  1.0234548846079456
Validation r2_score:  -9730183449466.066
Validation MAE:  0.8098785143066691
	Train Loss:  1.0234548846079456
	Validation MSE:  1.0399381135040648

Epoch 4/100
Learning rate =  0.009960573506572375
Train loss =  1.0035843926892485
Validation r2_score:  -15673607283216.965
Validation MAE:  0.8078354514571174
	Train Loss:  1.0035843926892485
	Validation MSE:  1.037282746655872

Epoch 5/100
Learning rate =  0.009938441702975677
Train loss =  0.9998852087445157
Vali